In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:

from dataclasses import dataclass
import asyncio
from agents.extensions.visualization import draw_graph
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)

In [153]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = Runner.run_sync(agent, "who is prime Minster of Pakistan in 2021",max_turns=1)
print(resp.final_output)

 Prime Minster of Pakistan in 2021 is imran khan 


MaxTurnsExceeded: Max turns (1) exceeded

In [150]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = Runner.run_streamed(agent, "who is prime Minster of Pakistan in 2021")
async for rep in resp.stream_events():
    print(rep)
print(resp.final_output)

AgentUpdatedStreamEvent(new_agent=Agent(name='tool agent', instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x00000141EC071400>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FunctionTool(name='get_weather', description='', params_json_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals

In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of Pakistan in 2021")
print(resp.final_output)

 Prime Minster of Pakistan in 2021 is imran khan 
پاکستان کے وزیر اعظم 2021 میں عمران خان تھے۔



In [148]:
from doctest import debug
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, Usage, function_tool,AgentHooks
class TurnCounter(AgentHooks):
    def __init__(self):
        self.turns = 0
        self.tool_calls = 0

    async def on_start(self, context: RunContextWrapper, agent: Agent):
        self.turns += 1
        print(f"\n🔄 Turn {self.turns} started (AI thinking)")

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool):
        self.tool_calls += 1
        print(f"🛠️ Tool `{tool.name}` called")
class CustomAgentHooks(AgentHooks):
    def __init__(self, display_name: str):
        self.event_counter = 0
        self.display_name = display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started")
    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started tool {tool.name}"
        )

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],hooks=TurnCounter(),
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of Pakistan in 2021")
print(resp.final_output)


🔄 Turn 1 started (AI thinking)
🛠️ Tool `find_primenister` called
 Prime Minster of Pakistan in 2021 is imran khan 
پاکستان کے وزیراعظم 2021 میں عمران خان تھے۔



In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool
def find_primenister(country:str,year:int=2025)->str:
    """ 
    This function logic find the prime minster of the country
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

 Prime Minster of pakistan in 20000 is imran khan 
Pakistan ke Prime Minister 20000 mein Imran Khan hain.



In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool
def find_primenister(country:str,year:int=2025)->str:
    """ 
    This function logic find the prime minster of the country
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2100:
        
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

UserError: Error running tool find_primenister: Year is too far in the future. Please enter a valid year.

In [100]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    # tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

UserError: Error running tool find_primenister: Year is too far in the future. Please enter a valid year.

In [50]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler(exception: Exception, ctx: RunContextWrapper) -> str:
    return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

⚠️ Tool error: RunContextWrapper(context=None, usage=Usage(requests=1, input_tokens=76, output_tokens=10, total_tokens=86))


In [101]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

⚠️ Tool error: Year is too far in the future. Please enter a valid year.


In [102]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    # tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

The year is too far in the future. Please enter a valid year.



In [104]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    pass
    # return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    # tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

I am unable to find the prime minister of Pakistan in 2021.



In [108]:
from agents import Agent, Runner, set_tracing_disabled, function_tool
 
import asyncio

set_tracing_disabled(disabled=False)

@function_tool(failure_error_function=None)
def extract_fun(a: str, b: int):
    """Add two numbers.

    Args:
        a: First argument.
        b: Second argument.
    """
    print(f"{a},{b}")
    print(f"{a},{b},{a+b}")
    return {a + b}

agent = Agent(
    name="test-agent",
    instructions="""
    You must call the `extract_fun` tool using exactly these values: a = 'arman', b = 'twenty four'.
    Do not ask the user for clarification. Do not rephrase. Just call the tool.
    """,
    tools=[extract_fun],
    model=model,
    
)

async def main():
    try:
        resp = await Runner.run(agent, "Please follow instructions strictly.")
        print(resp.final_output)
    except Exception as e:
        print(f"❌ Caught Exception: {type(e).__name__} - {e}")

asyncio.run(main())


arman,24
❌ Caught Exception: UserError - Error running tool extract_fun: can only concatenate str (not "int") to str


In [109]:
from agents import Agent, Runner, set_tracing_disabled, function_tool
import asyncio

set_tracing_disabled(disabled=False)

@function_tool(failure_error_function=None)
def extract_fun(a: str, b: int):
    """Add two numbers.

    Args:
        a: First argument.
        b: Second argument.
    """
    print(f"{a},{b},{a + b}")

agent = Agent(
    name="test-agent",
    instructions="""
    You must call the `extract_fun` tool. Do NOT correct your JSON or ask the user. 
    Provide your tool call in a single JSON object (no code fences).
    """,
    tools=[extract_fun],
    model=model, 
)

async def main():
    try:
        # Here we force the model to emit malformed JSON:
        resp = await Runner.run(agent,
            "Call extract_fun now with this exact text: { a: arman, b: }"
        )
        print("Final Output:", resp.final_output)
    except Exception as e:
        # This should be ModelBehaviorError
        print(f"❌ Caught Exception: {type(e).__name__} - {e}")

asyncio.run(main())


Final Output: I'm sorry, I cannot fulfill your request. The value for 'b' is missing.



In [81]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
 
from agents import function_tool

@function_tool()
def extract_fun(a:str, b:int):
    """Add two numbers.
   
    Args:
        a : First argument   .
        b : Second argument .
    """
    print(f"{a},{b},{a+b}")
    # return a + b


agent = Agent(
    name="test-agent",
    instructions="Always call the extract_fun tool.",
    tools=[extract_fun],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

try:
    resp = await Runner.run(agent, "  first value is name and its value is arman and second value is age and its value is 24")

    print(resp.final_output)
except Exception as e:
    print("❌ Tool failed:", type(e).__name__, "-", e)    

An error occurred while running the tool. Please try again. Error: can only concatenate str (not "int") to str


In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
 
@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)
try:
    resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
    print(resp.final_output)
except Exception as e:
    print("❌ Tool failed:", type(e).__name__, "-", e)    

In [53]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    pass
    # return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

UserError: Error running tool find_primenister: Year is too far in the future. Please enter a valid year.

In [38]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    # if year > 2100:
    #     raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

 Prime Minister of pakistan in 20000 is Imran Khan 
Prime Minister of pakistan in 20000 is Imran Khan


In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool(name_override='primenister')
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of india in 2021")
print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         # Output: greet
    print(tool.description) 

 Prime Minster of india in 2021 is imran khan 
بھارت کے وزیر اعظم 2021 میں عمران خان ہیں۔



In [ ]:
 # Output: Greet the user by name.

get_weather

primenister



In [19]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''Find the priminter of the country'''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         # Output: greet
    print(tool.description) 

get_weather

primenister
This function is used to find the prime Minster of country


In [13]:
from agents import Agent, Runner, set_tracing_disabled,function_tool
import json
set_tracing_disabled(disabled=False)

@function_tool()
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''Find the priminter of the country'''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         # Output: greet
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

get_weather

{
  "properties": {
    "city": {
      "title": "City",
      "type": "string"
    }
  },
  "required": [
    "city"
  ],
  "title": "get_weather_args",
  "type": "object",
  "additionalProperties": false
}
primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [29]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country',use_docstring_info=False)
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_primenister

{
  "properties": {
    "country": {
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "find_primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [113]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country' )
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from pydantic import BaseModel
from dataclasses import dataclass
from typing_extensions import TypedDict
from pydantic import BaseModel,Field

set_tracing_disabled(disabled=False)
@dataclass
class custom_class:
    name: str = Field(description="name of the custom_class")
    
   

 
@function_tool
def find_president(country:str,cus:custom_class,year:str=2025,)->str:
    '''
    Find the president of the country
        
    Args:
        country (int): Name of the country
        year (int): Year of the country
        cus (custom_class): custom object 
        '''
    resp =  f"prisident of {country} in {year} is imran khan"
    print(f"\033[31m prisident of {country} in {year} is imran khan\033[0m ")
    return resp 
 
agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[ find_president
    ],
    model=model,
) 
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_president
Find the president of the country
{
  "$defs": {
    "custom_class": {
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "cus": {
      "description": "custom object\nname (str): The name of the person to include.",
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": 2025,
      "description": "Y

In [137]:
from typing import Literal

def get_status(code: Literal["success", "error"]) -> str:
    return f"Status: {code}"

# get_status("success")  
get_status(1)      


'Status: 1'

In [130]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from pydantic import BaseModel
from dataclasses import dataclass
from typing_extensions import TypedDict
from pydantic import BaseModel,Field

set_tracing_disabled(disabled=False)

class custom_class(BaseModel):
    name: str = Field(description="name of the custom_class")
    
   

 
@function_tool
def find_president(country:str,cus:custom_class,year:str=2025,)->str:
    '''
    Find the president of the country
        
    Args:
        country (int): Name of the country
        year (int): Year of the country
        cus (custom_class): custom object 
        '''
    resp =  f"prisident of {country} in {year} is imran khan"
    print(f"\033[31m prisident of {country} in {year} is imran khan\033[0m ")
    return resp 
 
agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[ find_president
    ],
    model=model,
) 
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_president
Find the president of the country
{
  "$defs": {
    "custom_class": {
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "cus": {
      "description": "custom object",
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
    

In [134]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from pydantic import BaseModel
from dataclasses import dataclass
from typing_extensions import TypedDict
from pydantic import BaseModel,Field

set_tracing_disabled(disabled=False)

class custom_class(BaseModel):
    name: str = Field(description="name of the custom_class")
    
   

 
@function_tool
def find_president(country:str,cus:custom_class,year:str=2025,)->str:
    '''
    Find the president of the country
        
    Args:
        country (int): Name of the country
        year (int): Year of the country
        cus (custom_class): custom object 
        '''
    resp =  f"prisident of {country} in {year} is imran khan"
    print(f"\033[31m prisident of {country} in {year} is imran khan\033[0m ")
    return resp 
 
agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[ find_president
    ],
    model=model,
) 
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_president
Find the president of the country
{
  "$defs": {
    "custom_class": {
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "cus": {
      "description": "custom object",
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
    

In [ ]:
from typing_extensions import TypedDict
from agents import function_tool, Agent
import json
 
class Custom(TypedDict):
    name: str

@function_tool
def find_president(
    country: str,
    custom: Custom,
    year: str = "2025"
) -> str:
    """
    Find the president of the country.

    Args:
        country (str): Name of the country
        custom (Custom): Extra info
        year (str): Year (default 2025)
    """
    return f"President of {country} in {year} with {custom['name']}"

# 2) Register and inspect
agent = Agent(name="tool-agent", tools=[find_president], model=model)
for tool in agent.tools:
    print(json.dumps(tool.params_json_schema, indent=2))


{
  "$defs": {
    "Custom": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "Custom",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "custom": {
      "description": "Extra info",
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "Custom",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": "2025",
      "description": "Year (default 2025)",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "custom",
    "year"
  ],
  "title": "find_president_args",
  "type": "object",
  "additionalProperties": 

In [35]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: ''
        year: str
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "''",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "str",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [30]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

In [18]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from typing_extensions import  TypedDict
set_tracing_disabled(disabled=False)
class Location(TypedDict):
    lat: float
    long: float

@function_tool()
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,fetch_weather,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}
fetch_weather
Fetch the weather for a given location.
{
  "$defs": {
    "Location": {
      "properties": {
        "lat": {
          "title": "Lat",
          "type": "number"
        },
        "long": {
          "title": "Long",
          "type": "number"
        }
      },
      "required": [
        "lat",
        "long"
      ],
      "title": "Location",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "location": {
      "description": "The location to fetch the 

In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from typing_extensions import  TypedDict
set_tracing_disabled(disabled=False)
class Location(TypedDict):
    lat: float
    long: float

@function_tool()
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location (str): The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        this function finds the primenister of the country
    args:
        country (str): Name of the country
        year (str): Year of the country
    Returns:
        str : country prime Minster    
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister, 
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [26]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from typing_extensions import  TypedDict
set_tracing_disabled(disabled=False)
class Location(TypedDict):
    lat: float
    long: float

@function_tool()
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location (str): The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

@function_tool(name_override='primenister')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        this function finds the primenister of the country
        
    args:
        country (str): Name of the country
        year (str): Year of the country
    Returns:
        str : country prime Minster    
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister, 
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
Find the priminter of the country
    this function finds the primenister of the country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [4]:
import json
from typing import Any
from pydantic import BaseModel
from agents import RunContextWrapper, FunctionTool, Agent
 
def do_some_work(data: str) -> str:
    return "done" 
class FunctionArgs(BaseModel):
    username: str
    age: int 
async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")
 
custom_tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)
 
agent = Agent(
    name="CustomToolAgent",
    tools=[custom_tool],
)
 
for tool in agent.tools:
    print("🛠 Tool Name:", tool.name)
    print("📝 Description:", tool.description)
    print("📦 Input Schema:")
    print(json.dumps(tool.params_json_schema, indent=2))


🛠 Tool Name: process_user
📝 Description: Processes extracted user data
📦 Input Schema:
{
  "properties": {
    "username": {
      "title": "Username",
      "type": "string"
    },
    "age": {
      "title": "Age",
      "type": "integer"
    }
  },
  "required": [
    "username",
    "age"
  ],
  "title": "FunctionArgs",
  "type": "object"
}


In [ ]:
import json
from typing import Any
from pydantic import BaseModel
from agents import RunContextWrapper, FunctionTool, Agent
 
def do_some_work(data: str) -> str:
    return "done" 
class FunctionArgs(BaseModel):
    username: str
    age: int 
    
class newFunctionArgs(BaseModel):
    username: str
    age: int 
async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")
 
custom_tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)
 
agent = Agent(
    name="CustomToolAgent",
    tools=[custom_tool],
)
 
for tool in agent.tools:
    print("🛠 Tool Name:", tool.name)
    print("📝 Description:", tool.description)
    print("📦 Input Schema:")
    print(json.dumps(tool.params_json_schema, indent=2))


ModuleNotFoundError: No module named 'griffe.loader'